In [119]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain import hub
from langchain.agents import AgentExecutor,create_openai_tools_agent,create_react_agent,create_tool_calling_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.prompts import ChatPromptTemplate

In [109]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [123]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash-002")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [111]:
wikipediaAPIWrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=100)
tools=[WikipediaQueryRun(api_wrapper=wikipediaAPIWrapper)]

In [112]:
prompt=hub.pull(owner_repo_commit="hwchase17/react")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [73]:
tools[0].name

'wikipedia'

<h3>Will Give Error</h3>

In [74]:
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [77]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nAction: wikipedia"})



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I need to look up Leonardo DiCaprio on Wikipedia to find his full name, which should include his middle name.
Action Input: Leonardo DiCaprio`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE

In [41]:
print(prompt.to_json()['kwargs']['template'])

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [47]:
# We can change the Prompt and agent for it to work
prompt=hub.pull("hwchase17/openai-tools-agent")
agent=create_openai_tools_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [48]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nThought\n\nAction: Wikipedia"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Leonardo DiCaprio'}`


Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːprjo]; born November 1Leonardo DiCaprio's middle name is Wilhelm.

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?\n\nThought\n\nAction: Wikipedia",
 'output': "Leonardo DiCaprio's middle name is Wilhelm."}

In [49]:
prompt.to_json()['kwargs']['messages']

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [78]:
# Else we can Parse it
prompt=hub.pull(owner_repo_commit="hwchase17/react")
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True,handle_parsing_errors=True)

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [80]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought"})



> Entering new AgentExecutor chain...
I need to look up Leonardo DiCaprio's full name to find out his middle name.
Action Input: Leonardo DiCaprioInvalid Format: Missing 'Action:' after 'Thought:I noticed that I made a mistake in the format. I should input the action next.
Action: wikipedia
Action Input: Leonardo DiCaprioPage: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːprjo]; born November 1From the Wikipedia summary, I found that Leonardo DiCaprio's middle name is Wilhelm.
Final Answer: Wilhelm

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought",
 'output': 'Wilhelm'}

<h3>Apply Custom Error Messages</h3>

In [92]:
parsingError="Check your output and make sure it conforms, use the Action/Action Input syntax"
prompt=hub.pull(owner_repo_commit="hwchase17/react")
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True,handle_parsing_errors=parsingError)

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [93]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought"})



> Entering new AgentExecutor chain...
Could not parse LLM output: `I need to look up information about Leonardo DiCaprio.
Action Input: Leonardo DiCaprio`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURECheck your output and make sure it conforms, use the Action/Action Input syntaxThe Wikipedia page on Leonardo DiCaprio should contain information about his full name.

Action: wikipedia
Action Input: Leonardo DiCaprioPage: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːprjo]; born November 1The Wikipedia page lists Leonardo DiCaprio's full name as "Leonardo Wilhelm DiCaprio". 
Final Answer: Leonardo DiCaprio's middle name is Wilhelm.

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought",
 'output': "Leonardo DiCaprio's middle name is Wilhelm."}

<h3> Apply Custom Error Function</h3>

In [94]:
def _handleError(error):
    return str(error)
prompt=hub.pull(owner_repo_commit="hwchase17/react")
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True,handle_parsing_errors=_handleError)

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [95]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought"})



> Entering new AgentExecutor chain...
Could not parse LLM output: `I need to find information about Leonardo DiCaprio's full name, specifically his middle name. The best course of action would be to search for Leonardo DiCaprio on Wikipedia.

Action Input: Leonardo DiCaprio
`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURECould not parse LLM output: `I need to find information about Leonardo DiCaprio's full name, specifically his middle name. The best course of action would be to search for Leonardo DiCaprio on Wikipedia.

Action Input: Leonardo DiCaprio
`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURECould not parse LLM output: `I need to find the middle name of Leonardo DiCaprio from the information provided.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURECould not parse LLM output: `I need to find the 

{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia\n\nThought",
 'output': 'Wilhelm'}

<h3>Access Intermediate Steps</h3>

In [128]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a helpful assistant."),
        ("placeholder","{chat_history}"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)
agent=create_openai_tools_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True,return_intermediate_steps=True,handle_parsing_errors=True)

In [129]:
agentExecutor.invoke(input={"input":"What is Leo DiCaprio's middle name?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Leonardo DiCaprio'}`


Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːprjo]; born November 1Leonardo DiCaprio's middle name is Wilhelm.

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?",
 'output': "Leonardo DiCaprio's middle name is Wilhelm.",
 'intermediate_steps': [(ToolAgentAction(tool='wikipedia', tool_input={'query': 'Leonardo DiCaprio'}, log="\nInvoking: `wikipedia` with `{'query': 'Leonardo DiCaprio'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_qLhXoZgVlbUndGxdc3BgF376', 'function': {'arguments': '{"query":"Leonardo DiCaprio"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-a30f126d-cdb3-4b2d-ab43-8de173f7a9e2', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Leonardo DiCaprio'}, 'id': 'call_qLhXoZgVlbUndGxdc3BgF376', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'wikipedia', 'args': '{"query":"Leonardo DiCaprio"}', 'id': 'call_qLhXoZgVlbUndGxdc3BgF376', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_qLhXoZgVlbUndGxdc3BgF376'),
   'Pa